class-1 fraud

In [70]:
import pandas as pd
import numpy as np
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import StratifiedKFold
import lightgbm as lgbm
from sklearn.metrics import roc_auc_score
import optuna
from optuna.samplers import TPESampler



In [71]:
train=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')
print(train.shape,test.shape)

(219129, 32) (146087, 31)


In [72]:
train

,id,Time,V1,V2,V3,V4,V5,V6,V7,V8,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0,0.0,2.074329,-0.129425,-1.137418,0.412846,-0.192638,-1.210144,0.110697,-0.263477,...,-0.334701,-0.887840,0.336701,-0.110835,-0.291459,0.207733,-0.076576,-0.059577,1.98,0
1,1,0.0,1.998827,-1.250891,-0.520969,-0.894539,-1.122528,-0.270866,-1.029289,0.050198,...,0.054848,-0.038367,0.133518,-0.461928,-0.465491,-0.464655,-0.009413,-0.038238,84.00,0
2,2,0.0,0.091535,1.004517,-0.223445,-0.435249,0.667548,-0.988351,0.948146,-0.084789,...,-0.326725,-0.803736,0.154495,0.951233,-0.506919,0.085046,0.224458,0.087356,2.69,0
3,3,0.0,1.979649,-0.184949,-1.064206,0.120125,-0.215238,-0.648829,-0.087826,-0.035367,...,-0.095514,-0.079792,0.167701,-0.042939,0.000799,-0.096148,-0.057780,-0.073839,1.00,0
4,4,0.0,1.025898,-0.171827,1.203717,1.243900,-0.636572,1.099074,-0.938651,0.569239,...,0.099157,0.608908,0.027901,-0.262813,0.257834,-0.252829,0.108338,0.021051,1.00,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
219124,219124,120580.0,1.891079,-1.272908,-3.783908,-1.388939,2.012789,2.666080,0.151740,0.401934,...,-0.181369,-0.456538,-0.069571,0.756765,0.244479,-0.147566,-0.054725,-0.044588,198.65,0
219125,219125,120580.0,0.139724,0.948649,-2.913655,-2.184829,1.883716,-1.056824,1.725624,0.018089,...,0.491469,1.478823,-0.085398,-0.091409,-1.053488,0.467570,0.358918,0.294735,24.00,0
219126,219126,120580.0,2.058343,-0.038993,-1.928553,0.330117,0.270127,-0.735664,-0.173878,0.144823,...,-0.306640,-0.965783,0.307799,-0.021434,-0.343989,0.181065,-0.098387,-0.044064,1.79,0
219127,219127,120580.0,2.079227,-2.162389,-1.785833,-2.804889,0.552845,4.038013,-2.155900,1.023785,...,0.109909,0.590401,0.286621,0.675660,-0.510736,-0.090044,0.056749,-0.017126,88.00,0


In [73]:
from collections import Counter as c
c(train['Class'])

Counter({0: 218660, 1: 469})

In [74]:
test

,id,Time,V1,V2,V3,V4,V5,V6,V7,V8,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
0,219129,120580.0,2.115519,-0.691809,-1.305514,-0.685655,-0.641265,-0.764784,-0.924262,-0.023030,...,0.067367,0.241708,0.682524,0.037769,-0.546859,-0.123055,-0.084889,0.004720,-0.021944,29.95
1,219130,120580.0,1.743525,-1.681429,-0.547387,-1.061113,-0.695825,2.458824,-1.632859,1.073529,...,0.441788,0.543278,1.294571,0.309541,3.703925,-0.242579,0.068708,0.002629,0.064690,163.50
2,219131,120580.0,2.205568,-1.571445,-0.238965,-1.662517,-1.652324,-0.054701,-1.682064,0.105613,...,-0.366906,-0.131527,0.086623,0.291375,0.739087,-0.543006,-0.297813,0.043699,-0.037855,16.00
3,219132,120580.0,1.989728,-0.972909,-1.938259,-1.440129,-0.166855,-0.794048,0.252889,-0.399789,...,-0.049136,-0.080115,-0.010732,-0.038550,0.656830,0.343470,-0.627529,-0.024338,-0.036143,120.98
4,219133,120580.0,-1.943548,-1.668761,0.363601,-0.977610,2.684779,-2.037681,0.039709,-0.048895,...,0.391627,0.083389,-0.306918,0.247822,-0.391799,-0.790716,-0.025706,0.330758,0.335537,1.98
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146082,365211,172789.0,-0.661834,-0.622980,0.735580,-0.554067,0.869889,-0.389502,0.161226,0.069945,...,0.262408,-0.046068,-0.425692,0.253833,-1.076331,-0.999877,0.161938,0.149243,0.279708,94.81
146083,365212,172789.0,-0.098889,0.761900,0.340012,-0.589546,0.717091,-0.262765,0.726272,-0.070608,...,0.064640,0.441215,1.284819,-0.141790,0.646930,-0.419964,-0.561426,0.326700,0.255166,24.99
146084,365213,172789.0,-1.487918,1.769142,0.551671,0.361566,0.056347,0.084844,0.567545,-0.059899,...,0.792736,0.163500,1.287079,-0.186955,-0.349847,-0.161724,-0.213933,0.392745,0.070938,29.99
146085,365214,172790.0,-1.889979,-0.154869,0.068496,-1.906964,-1.743181,0.717490,0.981608,-0.209713,...,-0.513829,0.121457,0.764642,-0.217844,0.104599,-0.008906,-0.204957,-0.752326,-0.155728,314.00


In [75]:
test.isna().any().sum().sum()

0

In [76]:
train['hour'] = train['Time'] % (24 * 3600) // 3600
train['day'] = (train['Time'] // (24 * 3600)) % 7

test['hour'] = test['Time'] % (24 * 3600) // 3600
test['day'] = (test['Time'] // (24 * 3600)) % 7

In [77]:
train.drop(['id','Time'],axis=1,inplace=True)

In [78]:
train

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V23,V24,V25,V26,V27,V28,Amount,Class,hour,day
0,2.074329,-0.129425,-1.137418,0.412846,-0.192638,-1.210144,0.110697,-0.263477,0.742144,0.108782,...,0.336701,-0.110835,-0.291459,0.207733,-0.076576,-0.059577,1.98,0,0.0,0.0
1,1.998827,-1.250891,-0.520969,-0.894539,-1.122528,-0.270866,-1.029289,0.050198,-0.109948,0.908773,...,0.133518,-0.461928,-0.465491,-0.464655,-0.009413,-0.038238,84.00,0,0.0,0.0
2,0.091535,1.004517,-0.223445,-0.435249,0.667548,-0.988351,0.948146,-0.084789,-0.042027,-0.818383,...,0.154495,0.951233,-0.506919,0.085046,0.224458,0.087356,2.69,0,0.0,0.0
3,1.979649,-0.184949,-1.064206,0.120125,-0.215238,-0.648829,-0.087826,-0.035367,0.885838,-0.007527,...,0.167701,-0.042939,0.000799,-0.096148,-0.057780,-0.073839,1.00,0,0.0,0.0
4,1.025898,-0.171827,1.203717,1.243900,-0.636572,1.099074,-0.938651,0.569239,0.692665,-0.097495,...,0.027901,-0.262813,0.257834,-0.252829,0.108338,0.021051,1.00,0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
219124,1.891079,-1.272908,-3.783908,-1.388939,2.012789,2.666080,0.151740,0.401934,-1.102824,0.858158,...,-0.069571,0.756765,0.244479,-0.147566,-0.054725,-0.044588,198.65,0,9.0,1.0
219125,0.139724,0.948649,-2.913655,-2.184829,1.883716,-1.056824,1.725624,0.018089,-0.823494,-0.257933,...,-0.085398,-0.091409,-1.053488,0.467570,0.358918,0.294735,24.00,0,9.0,1.0
219126,2.058343,-0.038993,-1.928553,0.330117,0.270127,-0.735664,-0.173878,0.144823,0.849289,-0.136498,...,0.307799,-0.021434,-0.343989,0.181065,-0.098387,-0.044064,1.79,0,9.0,1.0
219127,2.079227,-2.162389,-1.785833,-2.804889,0.552845,4.038013,-2.155900,1.023785,-0.865242,1.536193,...,0.286621,0.675660,-0.510736,-0.090044,0.056749,-0.017126,88.00,0,9.0,1.0


In [79]:
# from sklearn.preprocessing import MinMaxScaler
# scaler=MinMaxScaler()
# train=scaler.fit_transform(train)


In [80]:
train=pd.DataFrame(train)
train

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V23,V24,V25,V26,V27,V28,Amount,Class,hour,day
0,2.074329,-0.129425,-1.137418,0.412846,-0.192638,-1.210144,0.110697,-0.263477,0.742144,0.108782,...,0.336701,-0.110835,-0.291459,0.207733,-0.076576,-0.059577,1.98,0,0.0,0.0
1,1.998827,-1.250891,-0.520969,-0.894539,-1.122528,-0.270866,-1.029289,0.050198,-0.109948,0.908773,...,0.133518,-0.461928,-0.465491,-0.464655,-0.009413,-0.038238,84.00,0,0.0,0.0
2,0.091535,1.004517,-0.223445,-0.435249,0.667548,-0.988351,0.948146,-0.084789,-0.042027,-0.818383,...,0.154495,0.951233,-0.506919,0.085046,0.224458,0.087356,2.69,0,0.0,0.0
3,1.979649,-0.184949,-1.064206,0.120125,-0.215238,-0.648829,-0.087826,-0.035367,0.885838,-0.007527,...,0.167701,-0.042939,0.000799,-0.096148,-0.057780,-0.073839,1.00,0,0.0,0.0
4,1.025898,-0.171827,1.203717,1.243900,-0.636572,1.099074,-0.938651,0.569239,0.692665,-0.097495,...,0.027901,-0.262813,0.257834,-0.252829,0.108338,0.021051,1.00,0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
219124,1.891079,-1.272908,-3.783908,-1.388939,2.012789,2.666080,0.151740,0.401934,-1.102824,0.858158,...,-0.069571,0.756765,0.244479,-0.147566,-0.054725,-0.044588,198.65,0,9.0,1.0
219125,0.139724,0.948649,-2.913655,-2.184829,1.883716,-1.056824,1.725624,0.018089,-0.823494,-0.257933,...,-0.085398,-0.091409,-1.053488,0.467570,0.358918,0.294735,24.00,0,9.0,1.0
219126,2.058343,-0.038993,-1.928553,0.330117,0.270127,-0.735664,-0.173878,0.144823,0.849289,-0.136498,...,0.307799,-0.021434,-0.343989,0.181065,-0.098387,-0.044064,1.79,0,9.0,1.0
219127,2.079227,-2.162389,-1.785833,-2.804889,0.552845,4.038013,-2.155900,1.023785,-0.865242,1.536193,...,0.286621,0.675660,-0.510736,-0.090044,0.056749,-0.017126,88.00,0,9.0,1.0


In [81]:
train

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V23,V24,V25,V26,V27,V28,Amount,Class,hour,day
0,2.074329,-0.129425,-1.137418,0.412846,-0.192638,-1.210144,0.110697,-0.263477,0.742144,0.108782,...,0.336701,-0.110835,-0.291459,0.207733,-0.076576,-0.059577,1.98,0,0.0,0.0
1,1.998827,-1.250891,-0.520969,-0.894539,-1.122528,-0.270866,-1.029289,0.050198,-0.109948,0.908773,...,0.133518,-0.461928,-0.465491,-0.464655,-0.009413,-0.038238,84.00,0,0.0,0.0
2,0.091535,1.004517,-0.223445,-0.435249,0.667548,-0.988351,0.948146,-0.084789,-0.042027,-0.818383,...,0.154495,0.951233,-0.506919,0.085046,0.224458,0.087356,2.69,0,0.0,0.0
3,1.979649,-0.184949,-1.064206,0.120125,-0.215238,-0.648829,-0.087826,-0.035367,0.885838,-0.007527,...,0.167701,-0.042939,0.000799,-0.096148,-0.057780,-0.073839,1.00,0,0.0,0.0
4,1.025898,-0.171827,1.203717,1.243900,-0.636572,1.099074,-0.938651,0.569239,0.692665,-0.097495,...,0.027901,-0.262813,0.257834,-0.252829,0.108338,0.021051,1.00,0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
219124,1.891079,-1.272908,-3.783908,-1.388939,2.012789,2.666080,0.151740,0.401934,-1.102824,0.858158,...,-0.069571,0.756765,0.244479,-0.147566,-0.054725,-0.044588,198.65,0,9.0,1.0
219125,0.139724,0.948649,-2.913655,-2.184829,1.883716,-1.056824,1.725624,0.018089,-0.823494,-0.257933,...,-0.085398,-0.091409,-1.053488,0.467570,0.358918,0.294735,24.00,0,9.0,1.0
219126,2.058343,-0.038993,-1.928553,0.330117,0.270127,-0.735664,-0.173878,0.144823,0.849289,-0.136498,...,0.307799,-0.021434,-0.343989,0.181065,-0.098387,-0.044064,1.79,0,9.0,1.0
219127,2.079227,-2.162389,-1.785833,-2.804889,0.552845,4.038013,-2.155900,1.023785,-0.865242,1.536193,...,0.286621,0.675660,-0.510736,-0.090044,0.056749,-0.017126,88.00,0,9.0,1.0


In [82]:
test

,id,Time,V1,V2,V3,V4,V5,V6,V7,V8,...,V22,V23,V24,V25,V26,V27,V28,Amount,hour,day
0,219129,120580.0,2.115519,-0.691809,-1.305514,-0.685655,-0.641265,-0.764784,-0.924262,-0.023030,...,0.682524,0.037769,-0.546859,-0.123055,-0.084889,0.004720,-0.021944,29.95,9.0,1.0
1,219130,120580.0,1.743525,-1.681429,-0.547387,-1.061113,-0.695825,2.458824,-1.632859,1.073529,...,1.294571,0.309541,3.703925,-0.242579,0.068708,0.002629,0.064690,163.50,9.0,1.0
2,219131,120580.0,2.205568,-1.571445,-0.238965,-1.662517,-1.652324,-0.054701,-1.682064,0.105613,...,0.086623,0.291375,0.739087,-0.543006,-0.297813,0.043699,-0.037855,16.00,9.0,1.0
3,219132,120580.0,1.989728,-0.972909,-1.938259,-1.440129,-0.166855,-0.794048,0.252889,-0.399789,...,-0.010732,-0.038550,0.656830,0.343470,-0.627529,-0.024338,-0.036143,120.98,9.0,1.0
4,219133,120580.0,-1.943548,-1.668761,0.363601,-0.977610,2.684779,-2.037681,0.039709,-0.048895,...,-0.306918,0.247822,-0.391799,-0.790716,-0.025706,0.330758,0.335537,1.98,9.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146082,365211,172789.0,-0.661834,-0.622980,0.735580,-0.554067,0.869889,-0.389502,0.161226,0.069945,...,-0.425692,0.253833,-1.076331,-0.999877,0.161938,0.149243,0.279708,94.81,23.0,1.0
146083,365212,172789.0,-0.098889,0.761900,0.340012,-0.589546,0.717091,-0.262765,0.726272,-0.070608,...,1.284819,-0.141790,0.646930,-0.419964,-0.561426,0.326700,0.255166,24.99,23.0,1.0
146084,365213,172789.0,-1.487918,1.769142,0.551671,0.361566,0.056347,0.084844,0.567545,-0.059899,...,1.287079,-0.186955,-0.349847,-0.161724,-0.213933,0.392745,0.070938,29.99,23.0,1.0
146085,365214,172790.0,-1.889979,-0.154869,0.068496,-1.906964,-1.743181,0.717490,0.981608,-0.209713,...,0.764642,-0.217844,0.104599,-0.008906,-0.204957,-0.752326,-0.155728,314.00,23.0,1.0


In [83]:
# train=train.iloc[:50000,:]

In [84]:
x1=train.columns
x1

Index(['V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11',
       'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20', 'V21',
       'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'Amount', 'Class',
       'hour', 'day'],
      dtype='object')

In [85]:
df=train[train['Class']==1]

In [86]:
df

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V23,V24,V25,V26,V27,V28,Amount,Class,hour,day
1681,0.845797,-1.063059,0.120133,-0.312570,-0.056194,1.944391,-0.661839,0.582721,-0.992366,0.564202,...,0.123612,-1.420935,-0.072559,0.293439,0.061270,0.006448,136.58,1,0.0,0.0
2039,-1.808089,1.794735,-2.574549,-1.944174,-4.093440,-1.360198,-8.837694,-3.865127,4.847707,-1.826233,...,-0.136751,0.739760,-0.692745,0.818293,0.745100,0.847648,2.00,1,0.0,0.0
2090,-0.368027,1.492913,0.653399,0.164478,-1.043918,0.766250,-2.717382,-6.944203,-0.938474,-1.243744,...,0.290564,0.528459,0.764303,-0.017768,0.110884,0.307835,0.12,1,0.0,0.0
3072,1.990241,-0.185837,-1.039174,0.086245,0.107037,-0.409425,0.016713,-0.126446,0.279865,0.184726,...,0.263769,0.779628,-0.261358,0.255895,-0.074999,-0.059828,23.31,1,0.0,0.0
3780,-0.738582,0.654464,0.652465,-0.139933,1.183689,-0.037131,0.638169,0.136502,-0.357541,-0.238928,...,0.012614,-0.311213,0.105798,-0.252849,0.049231,0.108966,4.99,1,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
218320,2.169813,-0.045807,-2.079652,0.207907,0.722241,-0.966274,0.564011,-0.381757,0.403626,0.264935,...,-0.041338,0.084528,0.363729,0.749486,-0.136626,-0.082399,6.99,1,9.0,1.0
218396,-11.419870,-39.517039,-15.773813,3.756988,-1.482355,-3.490795,-3.917593,0.865307,-3.497517,-4.095648,...,-0.072615,-0.664149,-0.494026,0.867453,-2.758080,0.185558,498.25,1,9.0,1.0
218404,1.708900,-1.003900,-1.531147,-0.173086,-0.430068,-0.367895,-0.073634,-0.231775,-0.067886,0.501141,...,-0.078629,0.009812,0.057939,-0.466157,0.049888,0.001680,229.99,1,9.0,1.0
218937,-1.330035,1.912497,0.058653,4.098485,0.555525,1.281339,-0.332319,1.016770,-1.751660,1.669879,...,0.287937,-0.391235,-0.871630,0.080706,0.179565,-0.092588,20.00,1,9.0,1.0


In [87]:
x=train.drop(['Class'],axis=1)
y=train['Class']

In [88]:
x

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V22,V23,V24,V25,V26,V27,V28,Amount,hour,day
0,2.074329,-0.129425,-1.137418,0.412846,-0.192638,-1.210144,0.110697,-0.263477,0.742144,0.108782,...,-0.887840,0.336701,-0.110835,-0.291459,0.207733,-0.076576,-0.059577,1.98,0.0,0.0
1,1.998827,-1.250891,-0.520969,-0.894539,-1.122528,-0.270866,-1.029289,0.050198,-0.109948,0.908773,...,-0.038367,0.133518,-0.461928,-0.465491,-0.464655,-0.009413,-0.038238,84.00,0.0,0.0
2,0.091535,1.004517,-0.223445,-0.435249,0.667548,-0.988351,0.948146,-0.084789,-0.042027,-0.818383,...,-0.803736,0.154495,0.951233,-0.506919,0.085046,0.224458,0.087356,2.69,0.0,0.0
3,1.979649,-0.184949,-1.064206,0.120125,-0.215238,-0.648829,-0.087826,-0.035367,0.885838,-0.007527,...,-0.079792,0.167701,-0.042939,0.000799,-0.096148,-0.057780,-0.073839,1.00,0.0,0.0
4,1.025898,-0.171827,1.203717,1.243900,-0.636572,1.099074,-0.938651,0.569239,0.692665,-0.097495,...,0.608908,0.027901,-0.262813,0.257834,-0.252829,0.108338,0.021051,1.00,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
219124,1.891079,-1.272908,-3.783908,-1.388939,2.012789,2.666080,0.151740,0.401934,-1.102824,0.858158,...,-0.456538,-0.069571,0.756765,0.244479,-0.147566,-0.054725,-0.044588,198.65,9.0,1.0
219125,0.139724,0.948649,-2.913655,-2.184829,1.883716,-1.056824,1.725624,0.018089,-0.823494,-0.257933,...,1.478823,-0.085398,-0.091409,-1.053488,0.467570,0.358918,0.294735,24.00,9.0,1.0
219126,2.058343,-0.038993,-1.928553,0.330117,0.270127,-0.735664,-0.173878,0.144823,0.849289,-0.136498,...,-0.965783,0.307799,-0.021434,-0.343989,0.181065,-0.098387,-0.044064,1.79,9.0,1.0
219127,2.079227,-2.162389,-1.785833,-2.804889,0.552845,4.038013,-2.155900,1.023785,-0.865242,1.536193,...,0.590401,0.286621,0.675660,-0.510736,-0.090044,0.056749,-0.017126,88.00,9.0,1.0


In [89]:
len(y)

219129

In [90]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [91]:
import numpy as np # linear algebra
import pandas as pd # data processing
import seaborn as sns # plotting tool
import matplotlib.pyplot as plt # plotting tool
from scipy import stats # statistical review
from sklearn.model_selection import train_test_split # train - test split
from sklearn.pipeline import Pipeline # data transformation
from sklearn.preprocessing import StandardScaler # data transformation
from sklearn.metrics import roc_auc_score # Competition metric
from xgboost import XGBClassifier # Xgboost
from sklearn.linear_model import LogisticRegression #Sklearn LR
from catboost import CatBoostClassifier #CatBoost
import xgboost as xgb
import lightgbm as lgb

In [ ]:
sub=pd.DataFrame(test['id'])
sub

,id
0,219129
1,219130
2,219131
3,219132
4,219133
...,...
146082,365211
146083,365212
146084,365213
146085,365214


In [ ]:
test.drop(['id','Time'],axis=1,inplace=True)

In [ ]:
mod1=model1.predict_proba(test)[:,1]
mod2=model2.predict_proba(test)[:,1]
mod3=model3.predict_proba(test)[:,1]
mod4=model4.predict_proba(test)[:,1]

In [ ]:

sub['Class']=1*mod1+0.3*mod2+0.2*mod3+0.4*mod4
sub

,id,Class
0,219129,0.001242
1,219130,0.000495
2,219131,0.000218
3,219132,0.001587
4,219133,0.000351
...,...,...
146082,365211,0.000506
146083,365212,0.001090
146084,365213,0.001778
146085,365214,0.000340


In [ ]:
sub.describe()

,id,Class
count,146087.000000,146087.000000
mean,292172.000000,0.001656
std,42171.828725,0.009113
min,219129.000000,0.000042
25%,255650.500000,0.000423
50%,292172.000000,0.000799
75%,328693.500000,0.001500
max,365215.000000,1.022761


In [ ]:
# sub['Class']=y_pred
# sub

In [ ]:
c(sub['Class'])

Counter({0.001241518038873055: 1,
         0.0004945519127380619: 1,
         0.00021823422210033537: 1,
         0.0015870484928208028: 1,
         0.0003512329340264279: 1,
         0.001363449399899013: 1,
         0.0005109336577473325: 1,
         0.0008597859559709104: 1,
         0.00048308850542645596: 1,
         0.0016518510930185568: 1,
         0.0008285320015840014: 1,
         0.00024203837898896004: 1,
         0.0009078229314454742: 1,
         0.0002531320478844777: 1,
         0.0002104195567621792: 1,
         0.00047751408249940413: 1,
         0.008841899386414234: 1,
         0.0005992971910261102: 1,
         0.0009493297116704063: 1,
         0.0024228747297638134: 1,
         0.00034292176535737236: 1,
         0.0017943939712381366: 1,
         0.0007371514149184739: 1,
         0.0022409777707203585: 1,
         0.0027874338716125397: 1,
         0.0004681950564181319: 1,
         0.00027854214208009157: 1,
         0.0007662233053200203: 1,
         0.001271

In [ ]:
sub.to_csv('sub.csv', index=False)